# GPS data and subsidence of the San Joaquin Valley

The example data we are going to use today is from continuously operating high-precision GPS stations that are operated by UNAVCO which is a non-profit university-governed consortium that facilitates geoscience research and education using geodesy. Let's get an introduction here: https://youtu.be/yxLMk120vMU

This data viewer gives a great summary velocity overview of the available GPS data: https://www.unavco.org/software/visualization/GPS-Velocity-Viewer/GPS-Velocity-Viewer.html

Let's start today by looking at data from a GPS station that is in Visalia California. Visalia is in the San Joaquin Valley between Fresno and Bakersfield.

https://www.unavco.org/instrumentation/networks/status/pbo/overview/P566

# Things we have covered

Today we will be focused on putting tools to use that we have used before with a couple new tricks related to dealing with time-series data (how to deal with dates). I will also include a bit more explanation of some of the things we are doing for the sake of review and leave in a few more error messages that we can discuss.

## Using `pandas` to import and filter data
From the first week of class we have used the pandas library to import and filter data. The DataFrame object has been the most common way we have dealt with data.

In [ ]:
import pandas as pd

One of the strengths of pandas is its ability to read and write different data types. For example, we have used the `pd.read_csv()` function to import .csv files throughout the course. This function can either be pointed to a file that is one your computer or a file that is posted on the internet. There are some online databases where you can use a url to access data of your choosing using a special format (API). We took this approach to get our birthquakes earlier in the course.

Let's import daily data since 2005 for the Visalia California GPS station. The data is in the North American tectonic plate (NAM14) reference frame which means that it takes the interior eastern part of North America functions as a fixed and stable point.

ftp://data-out.unavco.org/pub/products/position/P566/P566.cwu.nam14.csv

Let's enter this ftp address above.

In [ ]:
P566_GPS_data = pd.read_csv('ftp://data-out.unavco.org/pub/products/position/P566/P566.cwu.nam14.csv')

Whoops. There was an error. I kept this error in here as a reminder that I get errors all of the time. Remember that the errors are informative, but can also be a bit cryptic. In this case, it says "Expected 2 fields in line 10, saw 4." So it seems like there is a mismatch between the number of columns it is expecting and the number that there are. 

Let's look at the file: ftp://data-out.unavco.org/pub/products/position/P566/P566.cwu.nam14.csv

It turns out that there are a bunch of header lines and the header row that contains the column names needs to be specified (`header = 11`).

In [ ]:
P566_GPS_data = pd.read_csv('ftp://data-out.unavco.org/pub/products/position/P566/P566.cwu.nam14.csv', header=11)

We know how to take a peak at a DataFrame by applying the `.head()` function.

In [ ]:
P566_GPS_data.head()

We have done a lot where we have used extracted data from a single column. We have used the syntax `DataFrameName['column_name]`. It can be helpful to look at the available columns:

In [ ]:
P566_GPS_data.columns

Let first look at how this point is moving north with respect to stable North America. Notice that column names can be weird sometimes as some of the ones above have an extra space before them. If we don't put in that space, we won't be able to retrive the column values.

In [ ]:
P566_GPS_data[' North (mm)']

By themselves these data are pretty cool. It looks like Visalia has moved north relative to stable North America by ~130 mm (13 cm) over the past 14 years (the data starts in November 2005).

### Pandas timeseries

Pandas is good at dealing with time series data. We need to make sure that the data type of the 'Date' column is a time series

In [ ]:
P566_GPS_data['Date'][0]

In [ ]:
type(P566_GPS_data['Date'][0])

Right now, pandas thinks that the values in the data column are strings (a sequence of characters) rather than datetime values. We can convert them to be datetime values using `pd.to_datetime`.

In [ ]:
P566_GPS_data['Date'] = pd.to_datetime(P566_GPS_data['Date'])

In [ ]:
P566_GPS_data['Date'][0]

In [ ]:
type(P566_GPS_data['Date'][0])

## Making plots using `matplotlib`

We have relied on `matplotlib` to make plots throughout the course which we have imported as follows:

In [ ]:
import matplotlib.pyplot as plt

### Plotting with pandas

One thing that you can do using pandas once you have imported matplotlib that we haven't done very much is use built-in plotting functions on the DataFrame. In this case we can use `.plot`.

In [ ]:
P566_GPS_data.plot(x='Date', y=' North (mm)')

### Plotting using plt functions

We have made a number of different plot types using `matplotlib` such as `plt.hist()`, `plt.plot()` and `plt.scatter()`. When dealing with timedate values, one can use `plt.plot()`, but not `plt.scatter()`. Let's visualize both the north and east columns using `plt.plot()`. The data are from every day between Nov. 16, 2005 and now.

In [ ]:
P566_GPS_days = P566_GPS_data.index.values

In [ ]:
plt.plot(P566_GPS_data['Date'],P566_GPS_data[' North (mm)'])
plt.ylabel('north since start (mm)')
plt.xlabel('date')
plt.title('GPS data from station P566 (Visalia, CA)')
plt.show()

In [ ]:
plt.plot(P566_GPS_data['Date'],P566_GPS_data[' East (mm)'])
plt.ylabel('east since start (mm)')
plt.xlabel('date')
plt.title('GPS data from station P566 (Visalia, CA)')
plt.show()

What is going on with that drop midway through 2019? Let's take a look. I played around with what values were zoomed in that drop and found that `[4900:4950]` gave a good zoom-in:

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(P566_GPS_data['Date'][4900:4950],P566_GPS_data[' East (mm)'][4900:4950],'.')
plt.ylabel('east since start (mm)')
plt.xlabel('date')
plt.title('GPS data from station P566 (Visalia, CA)')
plt.tight_layout()
plt.show()

What happened on July 6, 2019?

https://earthquake.usgs.gov/earthquakes/eventpage/ci38457511/executive

![alt text](https://static.temblor.net/wp-content/uploads/2016/05/eastern-california-21.jpg)

## Calculations using `numpy` and `scipy`

We have used both the `numpy` and `scipy` libraries of functions to make numerical and statistical calculations. In particular we have put the numpy array data structure to work for us.

In [ ]:
import numpy as np

## Fitting a line with `np.polyfit()`

`np.polyfit()` can be used to calculate best fit lines (setting the degree (```deg```) to 1), or higher order curves (setting degree to 2 or higher) returning the slope and the intercept. We have put this to use in dealing with supernovae data and pCO$_2$ records. Let's put it to use here as well.

In [ ]:
np.polyfit(P566_GPS_data['Date'],P566_GPS_data[' North (mm)'],1)

Whoops. Again the error message is a bit cryptic, but it seems to be telling us that it can't deal with the data types. The np.polyfit function doesn't look to be able to deal with the datetime values. 

We can calculate days by making a new column in the data frame that is the 'date' value minus the initial date. This will be the number of days since the first date in the data set (Nov. 16 2005). How did I figure out how to do this properly? *I googled it and found an answer on stackoverflow.com.* Everyone from your classmates, to your professors, to professional software developers are reliant on seeking out help in order to figure out how to preform such technical computation tasks. 

In [ ]:
P566_GPS_data['days'] = (P566_GPS_data['Date'] - P566_GPS_data['Date'][0]).astype('timedelta64[D]')

Let's take a look at our DataFrame and make sure it has a new column `days` and that the column looks good.

In [ ]:
P566_GPS_data.head()

Now we can use `np.polyfit()` to do a linear regression between the days (`P566_GPS_data['days']`) and the distance traveled north (`P566_GPS_data[' North (mm)']`)

In [ ]:
np.polyfit(P566_GPS_data['days'],P566_GPS_data[' North (mm)'],1)

We can assign the best fitting slope and y-intercept from `np.polyfit()` to a variable (`P566_north_fit`).

In [ ]:
P566_north_fit = np.polyfit(P566_GPS_data['days'],P566_GPS_data[' North (mm)'],1)

In [ ]:
P566_north_fit

`P566_north_fit`  is an array of coefficients, where the first is the slope and the second is the y-intercept. We can get at that first element with square brackets.

In [ ]:
P566_north_fit[0]

**What are the units of this slope?**

*write your answer here*

 ### Make a plot and prediction with `np.polyval()`

We used the `np.polyval()` function back when we were analyzing the Hubble data and the Mauna Loa CO$_2$ data.

***Code for you to write***

**Use the function `np.polyval()` to plot a best-fit line for the data.** *Recall that you can feed in the linear model coefficients and the x-values (in this case the days) and calculate model y values (in this case north movement in mm).*

**Calculate and plot the residual.** *Recall that the residual is the difference between the actual data and the values obtained with the linear model.*

**Use the same function to predict how far north (relative to stable North America) the Visalia station will go in the next 10 years.** *There are 365.25 days in a year.*

### Developing the linear model with `stats.linregress`

We'd also like to know who well this model fits our data (i.e. how correlated the data are). The $R^{2}$ correlation coefficient can be helpful in this regard. $R^{2}$ is zero for uncorrelated data, and 1 for perfectly linear data (so no misfit between the model line and data). The scipy function `stats.linregress` can be used to implement the linear model and also return the correlation coefficient. 

In [ ]:
from scipy.stats import stats

In [ ]:
stats.linregress(P566_GPS_data['days'],P566_GPS_data[' North (mm)'])

In [ ]:
slope, intercept, rvalue, pvalue, stderr = stats.linregress(P566_GPS_data['days'],P566_GPS_data[' North (mm)'])

In [ ]:
rvalue**2

### Defining a function

When you may be doing a calculation more than once it is a good idea to define a function. Let's define a function that will take an east magnitude and a north magnetitude and return a direction between 0 and 360.

In [ ]:
def GPS_direction(east_magnitude, north_magnitude):
    direction_rad = np.arctan2(east_magnitude, north_magnitude)
    direction = np.rad2deg(direction_rad) % 360
    return direction

In [ ]:
GPS_direction(0,-1)

***Code for you to write***

**Calculate the slope for the east data. Use this slope and the slope for the north data to calculate the direction of the motion of the station using the `GPS_direction` function.**

## Making a map with `cartopy`

At the start of the course, we made a number of maps using the `cartopy` library. I have put in code below that will make a map showing the location of the P566 GPS station. I put in a number of bells and whistles including using terrain imagery tiles from Stamen and putting on text that labels the station location.

Let's define variables giving the latitude and longitude of the P566 GPS station.

In [ ]:
P566_lat = 36.32445
P566_lon = -119.22929

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.img_tiles import Stamen
import cartopy.feature as cfeature

plt.figure(figsize=(10,10))
imagery = Stamen('terrain-background')
ax = plt.axes(projection=imagery.crs)
ax.set_extent((-125, -114, 32, 42.5))
ax.add_image(imagery, 7)

states_provinces = cfeature.NaturalEarthFeature(category='cultural', 
                                                name='admin_1_states_provinces_lines',
                                                scale='50m', facecolor='none', linewidth=1)
ax.add_feature(states_provinces, edgecolor='black')

ax.scatter(P566_lon,P566_lat,transform=ccrs.Geodetic(),
           color='red',s=50)
ax.text(P566_lon,P566_lat,'P566\nstation ',transform=ccrs.Geodetic(),
           color='red',horizontalalignment='right',size=12)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
imagery = Stamen('terrain')
ax = plt.axes(projection=imagery.crs)
ax.set_extent((P566_lon-0.05, P566_lon+0.05, P566_lat-0.05, P566_lat+0.05))
ax.add_image(imagery, 13)

ax.scatter(P566_lon,P566_lat,transform=ccrs.Geodetic(),
           color='red',s=50)
ax.text(P566_lon,P566_lat,'P566\nstation ',transform=ccrs.Geodetic(),
           color='red',horizontalalignment='right',size=12)
plt.show()

**What is the main economic activity around P566? And what resources does that activity require?**

*write your answer here*

### Plot the vertical component of the GPS time-series

In [ ]:
plt.plot(P566_GPS_data['days'],P566_GPS_data[' Vertical (mm)'])
plt.ylabel('vertical since start (mm)')
plt.xlabel('day since start')
plt.title('GPS data from station P566 (Visalia, CA)')
plt.show()

**What do these data show? What is happening to the land surface? Why?**

https://earthobservatory.nasa.gov/images/89761/san-joaquin-valley-is-still-sinking

https://www.earthdate.org/californias-sinking-valley

Let's look at a subset of the data between 2006-01-01 and 2010-01-01. We have done a lot of this filtering using pandas. However, the syntax is hard to remember.

It can be helpful to remember how this is actually working under the hood. When we are passing in a conditional statement like `P566_GPS_data['Date'] >= '2006-01-01'` we are asking pandas to tell us, at every value in the `P566_GPS_data['Date']` column is it true or false that the date is greater than 2006-01-01? 

In [ ]:
P566_GPS_data['Date'] >= '2006-01-01'

The result is a list of true/false values. We then use these true/false values to filter the values in the DataFrame only returning those that are true. We can link multiple conditionals together with the `&` symbol such as in the example below `(P566_GPS_data['Date'] >= '2006-01-01') & (P566_GPS_data['Date'] < '2010-01-01')`.

In [ ]:
P566_GPS_06_09 = P566_GPS_data[(P566_GPS_data['Date'] >= '2006-01-01') & (P566_GPS_data['Date'] < '2010-01-01')]
P566_GPS_06_09.head()

In [ ]:
plt.plot(P566_GPS_06_09['Date'],P566_GPS_06_09[' Vertical (mm)'])
plt.ylabel('vertical since start (mm)')
plt.xlabel('date')
plt.title('GPS data from station P566 (Visalia, CA)')
plt.show()

## Estimating the slope and associated uncertainty

Like we did for the directional data when estimating the median and its uncertainty, we can get an estimate of the uncertainty of slope of the subsidence by bootstrap resampling the data a large number of times, and calculating the regression line through each bootstrapped dataset. This chapter in Inferential Thinking provides good explanations for this approach: https://www.inferentialthinking.com/chapters/16/Inference_for_Regression.html

We can collect all the slopes and plot their empirical histogram. We will use the `.sample()` function on the `P566_GPS_06_09` DataFrame and grab as many samples as the DataFrame is long (`len(P566_GPS_06_09)`) and replace the values each time they are sample (`replace=True`).

> The Bootstrap: Resampling from the Sample 

> What we do have is a large random sample from the population. As we know, a large random sample is likely to resemble the population from which it is drawn. This observation allows data scientists to lift themselves up by their own bootstraps: the sampling procedure can be replicated by sampling from the sample.

We need to put together a few different programming elements to make this work. Let's break them down.
- make a placeholder list that we with subsequently append values to: `slopes_06_09 = []`
- resample the DataFrame using `.sample`
- calculate a linear regression using `np.polyfit()`
- extract the slope which is the first element of what is returned from `np.polyfit()`
- append that value to the `slopes_06_09` list

In [ ]:
slopes_06_09 = []

resample = P566_GPS_06_09.sample(n=len(P566_GPS_06_09),replace=True)
fit = np.polyfit(resample['days'],resample[' North (mm)'],1)
slope = fit[0]
slopes_06_09.append(slope)

In [ ]:
slopes_06_09

But what we did above just does the resampling and slope fitting once. The idea of the bootstrap is that we are going to do it many many times.

To do this we will use a `for` loop. To loop through a set of code a specified number of times, we can use the `range()` function,
The `range()` function returns a sequence of numbers, starting from 0 by default, and increments by 1 (by default), and ends at a specified number. Anything that is indented under the for loop will then be repeated for the number of times that are in the range.

In [ ]:
for n in range(0,10):
    print(n)

For the bootstrap resampling and calculation of slope for the bootstrap resampled data, we want to do it many times. So let's do `for n in range(0,1000):`.  

In [ ]:
slopes_06_09 = []

for n in range(0,1000):
    resample = P566_GPS_06_09.sample(n=len(P566_GPS_06_09),replace=True)
    fit = np.polyfit(resample['days'],resample[' North (mm)'],1)
    slope = fit[0]
    slopes_06_09.append(slope)

We will now have a large number of slope values for the bootstrap resampled data. We can visualize them using `plt.hist()`.

In [ ]:
plt.hist(slopes_06_09)
plt.ylabel('number of values')
plt.xlabel('subsidence (mm/day)')
plt.show()

**Code for you to write**

There was a severe drought in California between 2013 and 2017

![alt text](https://upload.wikimedia.org/wikipedia/commons/1/1c/Drought_area_in_California.svg)
[https://upload.wikimedia.org/wikipedia/commons/1/1c/Drought_area_in_California.svg]

**Address the question: was the rate of land subsidence greater between 2013-01-01 and 2017-01-01 than it was between 2006-01-01 and 2010-01-01?** 

To do so:
- Filter the DataFrame to only include those years
- Use the bootstrap method to estimate the slope and its 95 percent confidence bounds (using the `np.percentile` function will help)
- Compare the the slopes between the two age ranges. Which one appears to be greater? Is this results significant or do they have overlapping confidence bounds? Plotting histograms of the bootstrap slope values is a good way to make this comparison.

*Bonus: fit a linear regression to the whole time series, calculate and plot the residuals. What does the residual correspond to?* 